In [1]:
from IPython.core.display import HTML
with open('../../style.css') as file:
    css = file.read()
HTML(css)

# Embedded Actions in <span style="font-variant:small-caps;">Antlr</span> Grammars

The *pure grammar* is stored in the file `Grammar.g4`.  On *Unix-like* operating
systems, the following command displays the file `Program.g4`.  

In [2]:
!cat -n Program.g4

     1	grammar Program;
     2	
     3	start: statement+ ; 
     4	
     5	statement
     6	    : IDENTIFIER ':=' expr ';' 
     7	    | expr ';'                 
     8	    ;
     9	
    10	expr: expr '+' product 
    11	    | expr '-' product 
    12	    | product
    13	    ;
    14	
    15	product 
    16	    : product '*' factor 
    17	    | product '/' factor 
    18	    | factor
    19	    ;
    20	
    21	factor
    22	    : 'sqrt' '(' expr ')'
    23	    | '(' expr ')'        
    24	    | FLOAT               
    25	    | IDENTIFIER          
    26	    ;
    27	
    28	IDENTIFIER: [a-zA-Z][a-zA-Z0-9_]*;
    29	FLOAT     : '0'([.][0-9]+)?
    30	          | [1-9][0-9]*([.][0-9]+)?;
    31	WS        : [ \t\n\r] -> skip; 


On a *Windows* operating system the cell given below prints the contend of
the file `Program.g4`.

In [ ]:
!type Program.g4

The grammar shown above has no *semantic actions* (with the exception of the `skip` action). 
We extend this grammar now with *semantic actions* so that we can actually compute something.
This grammar is stored in the file `Calculator.g4`.  It describes a language for a 
*symbolic calculator*: This calculator is able to evaluate arithmetic expressions and, furthermore, 
where we can store the results of our computations in variables.  

In [3]:
!cat -n Calculator.g4

     1	grammar Calculator;
     2	
     3	@header {
     4	import math
     5	}
     6	
     7	start: statement+ ; 
     8	
     9	statement
    10	    : IDENTIFIER ':=' expr ';' {self.Values[$IDENTIFIER.text] = $expr.result}
    11	    | expr ';'                 {print($expr.result)                         }
    12	    ;
    13	
    14	expr returns[result]
    15	    : e=expr '+' p=product {$result = $e.result + $p.result}
    16	    | e=expr '-' p=product {$result = $e.result - $p.result}
    17	    | p=product            {$result = $p.result            }
    18	    ;
    19	
    20	product returns[result]
    21	    : p=product '*' f=factor {$result = $p.result * $f.result}
    22	    | p=product '/' f=factor {$result = $p.result / $f.result}
    23	    | f=factor               {$result = $f.result            }
    24	    ;
    25	
    26	factor returns[result]
    27	    : 'sqrt' '(' expr ')' {$result = math.sqrt($expr.result)      }
    28	    | '(' expr ')'        {$result = $exp

In [ ]:
!type Calculator.g4

First, we have to generate both the scanner and the parser.  

In [4]:
!antlr4 -Dlanguage=Python3 Calculator.g4

We can use the system command `ls` to see which files have been generated by <span style="font-variant:small-caps;">Antlr</span>.
If you are using a windows system you have to use the command `dir` instead.

In [5]:
!ls -l

total 144
-rw-r--r--@ 1 karlstroetmann  staff   1106 Oct 31  2020 Calculator.g4
-rw-r--r--@ 1 karlstroetmann  staff   2644 Nov  9 16:15 Calculator.interp
-rw-r--r--@ 1 karlstroetmann  staff  13539 Nov  9 16:09 Calculator.ipynb
-rw-r--r--@ 1 karlstroetmann  staff    150 Nov  9 16:15 Calculator.tokens
-rw-r--r--@ 1 karlstroetmann  staff   3099 Nov  9 16:15 CalculatorLexer.interp
-rw-r--r--@ 1 karlstroetmann  staff   3684 Nov  9 16:15 CalculatorLexer.py
-rw-r--r--@ 1 karlstroetmann  staff    150 Nov  9 16:15 CalculatorLexer.tokens
-rw-r--r--@ 1 karlstroetmann  staff   1780 Nov  9 16:15 CalculatorListener.py
-rw-r--r--@ 1 karlstroetmann  staff  20524 Nov  9 16:15 CalculatorParser.py
-rw-r--r--@ 1 karlstroetmann  staff    524 Oct 31  2020 Program.g4


In [ ]:
!dir

The files `CalculatorLexer.py` and `CalculatorParser.py` contain the generated scanner and parser, respectively.  We have to import these files.  Furthermore, the runtime of 
<span style="font-variant:small-caps;">Antlr</span>
needs to be imported.

In [6]:
from CalculatorLexer  import CalculatorLexer
from CalculatorParser import CalculatorParser
import antlr4

Let us parse and evaluate the input that we read from a prompt.

In [7]:
def main():
    parser        = CalculatorParser(None) # generate parser without lexer
    parser.Values = {}
    line          = input('> ')
    while line != '':
        input_stream  = antlr4.InputStream(line)
        lexer         = CalculatorLexer(input_stream)
        token_stream  = antlr4.CommonTokenStream(lexer)
        parser.setInputStream(token_stream)
        parser.start()
        line = input('Input an arithmetic expression: ')
    return parser.Values

In [8]:
main()

> 1 + 2 * 3;
7.0
Input an arithmetic expression: x := 3;
Input an arithmetic expression: y := 4;
Input an arithmetic expression: z := sqrt(x*x+y*y);
Input an arithmetic expression: z;
5.0
Input an arithmetic expression: 


{'x': 3.0, 'y': 4.0, 'z': 5.0}

On a *Unix-like* system, i.e. on *MacOs* or *Linux* the following cell removes the
generated files.

In [9]:
!rm *.py *.tokens *.interp
!rm -r __pycache__/
!ls -l

total 48
-rw-r--r--@ 1 karlstroetmann  staff   1106 Oct 31  2020 Calculator.g4
-rw-r--r--@ 1 karlstroetmann  staff  14518 Nov  9 16:17 Calculator.ipynb
-rw-r--r--@ 1 karlstroetmann  staff    524 Oct 31  2020 Program.g4


On *Windows*, the following cell removes the generated files.

In [ ]:
!del *.py *.tokens *.interp
!rmdir __pycache__ /s /q
!dir